In [29]:
import matplotlib.pyplot as plt 
import numpy as np
import json
import pandas as pd
import plotly.express as px 
import os
from tabulate import tabulate

In [30]:
print(os.getcwd())

/Users/cd8607/Documents/Test_Potentials/Data/Defect Analysis


In [31]:
# os.chdir('../Data/Defect Analysis/')

In [32]:
with open('point_defects.json', 'r') as file:
    data = json.load(file)

In [33]:
perfect = data['V_0H_0He_0']['energy_opt']

for key in data:
    data[key]['energy_opt'] -= perfect

In [34]:
df = pd.DataFrame(data)

df_energy = df.iloc[0]

print(df_energy)

binding = {}

V_0H_0He_0          0.0
V_0H_1He_0     -1.32504
V_0H_2He_0    -2.818034
V_0H_3He_0    -4.216612
V_0H_4He_0    -5.765242
                ...    
V_1H_6He_2     2.964112
V_1H_6He_3      6.04467
V_1H_6He_4     9.336843
V_1H_6He_5    12.727785
V_1H_6He_6    16.998208
Name: energy_opt, Length: 111, dtype: object


In [35]:
def find_binding(binding, n_vac, n_int, add_element, title):

    add_int = df_energy['V_0H_%dHe_%d' % (add_element[0], add_element[1])]

    init_config = []
    final_config = []

    for [n_h, n_he] in n_int:

        init_config.append(df_energy['V_%dH_%dHe_%d' % (n_vac, n_h, n_he)])

        final_config.append(df_energy['V_%dH_%dHe_%d' % (n_vac, n_h + add_element[0], n_he + add_element[1])])

    init_config = np.array(init_config)

    final_config = np.array(final_config)

    print(init_config, final_config)
    binding['\ce{%s}' % title] = add_int + init_config - final_config
    
    return binding

In [36]:
binding = {}

for vac in range(2):
    for i in range(1):
        binding = find_binding(binding, vac, np.hstack([i*np.ones([6,1],dtype=int), np.arange(6).reshape(6,1)]),  [0,1], 'V%dH%dHe_x + He' % (vac, i))
        binding = find_binding(binding, vac, np.hstack([np.arange(6).reshape(6,1),  i*np.ones([6,1],dtype=int)]), [1,0], 'V%dH_xHe%d + H' % (vac, i))


[ 0.          6.68583835 12.45439473 17.77821183 22.92298795 27.90669161] [ 6.68583835 12.45439473 17.77821183 22.92298795 27.90669161 32.92785599]
[ 0.         -1.3250399  -2.81803387 -4.21661217 -5.76524248 -7.09386288] [-1.3250399  -2.81803387 -4.21661217 -5.76524248 -7.09386288 -8.46482618]
[12.58417698 13.87799298 17.20424602 20.76608664 24.35024418 28.663684  ] [13.87799298 17.20424602 20.76608664 24.35024418 28.663684   32.61765653]
[12.58417698 10.02525613  7.35054084  5.05042406  2.74490755  0.64389763] [10.02525613  7.35054084  5.05042406  2.74490755  0.64389763 -1.16782998]


In [37]:
binding = {}

for vac in range(2):
    for i in range(6):
        binding = find_binding(binding, vac, np.hstack([i*np.ones([6,1],dtype=int), np.arange(6).reshape(6,1)]), [1,0], 'V%dH%dHe_x + H' % (vac, i))


[ 0.          6.68583835 12.45439473 17.77821183 22.92298795 27.90669161] [-1.3250399   4.63380346 10.05065766 15.09867276 20.03629331 25.03476874]
[-1.3250399   4.63380346 10.05065766 15.09867276 20.03629331 25.03476874] [-2.81803387  2.51019043  7.79133345 12.51261373 17.37728927 22.08673103]
[-2.81803387  2.51019043  7.79133345 12.51261373 17.37728927 22.08673103] [-4.21661217  0.51044882  5.82316515 10.50357243 15.17916091 19.90638678]
[-4.21661217  0.51044882  5.82316515 10.50357243 15.17916091 19.90638678] [-5.76524248 -1.1703989   3.55705358  8.48640969 13.19401573 17.53117076]
[-5.76524248 -1.1703989   3.55705358  8.48640969 13.19401573 17.53117076] [-7.09386288 -3.05859614  1.55986241  6.44121079 11.168501   15.50381558]
[-7.09386288 -3.05859614  1.55986241  6.44121079 11.168501   15.50381558] [-8.46482618 -4.63862266 -0.12285935  4.35905332  9.05415816 13.26602375]
[12.58417698 13.87799298 17.20424602 20.76608664 24.35024418 28.663684  ] [10.02525613 11.32409983 14.60139617 1

In [38]:
df = pd.DataFrame(binding)

In [39]:
latex_content = ''

for column_name, column_data in df.iteritems():
    # Convert the column data to a LaTeX table
    latex_table = tabulate(pd.DataFrame(column_data), headers=['x', 'this work'], tablefmt='latex', floatfmt='.2f')
    
    # Add the LaTeX table to the content string
    latex_content += f"\\subsubsection*{{{column_name}}}\n"
    latex_content += latex_table + "\n\n"

/var/folders/s9/zg8t26bx2xg094_h3_f559v40000gp/T/ipykernel_15269/965674865.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


In [40]:
with open("output2.tex", "w") as latex_file:
    latex_file.write(latex_content)

print(df)

   \ce{V0H0He_x + H}  \ce{V0H1He_x + H}  \ce{V0H2He_x + H}  \ce{V0H3He_x + H}  \
0           0.000000           0.167954           0.073538           0.223590   
1           0.726995           0.798573           0.674702           0.355808   
2           1.078697           0.934284           0.643128           0.941072   
3           1.354499           1.261019           0.684001           0.692123   
4           1.561655           1.333964           0.873088           0.660105   
5           1.546883           1.622998           0.855304           1.050176   

   \ce{V0H4He_x + H}  \ce{V0H5He_x + H}  \ce{V1H0He_x + H}  \ce{V1H1He_x + H}  \
0           0.003580           0.045923           1.233881           1.349675   
1           0.563157           0.254987           1.228853           1.352782   
2           0.672151           0.357682           1.277810           1.354831   
3           0.720159           0.757118           1.253669           1.444996   
4           0.700475       

In [41]:
lst = np.hstack([np.zeros([5,1],dtype=int), np.arange(5).reshape(5,1)])
for i in lst:
    print(i)

[0 0]
[0 1]
[0 2]
[0 3]
[0 4]
